In [5]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train.csv', 
                   skiprows=range(1,144708152), nrows=181878211-144708152,
                   header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))


data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

#print('unique days in sample:' ,data['day'].unique())

len read: 37170059


In [ ]:
#print('record counts in days in sample:', data[['day','channel']].groupby(['day']).count())

mean = data[['ip', 'app', 'device', 'os','is_attributed']].groupby(by=['ip', 'app', 'device', 'os'])[['is_attributed']].mean().rename(columns={'is_attributed':'mean'})
count = data[['ip', 'app', 'device', 'os','is_attributed']].groupby(by=['ip', 'app', 'device', 'os'])[['is_attributed']].count().rename(columns={'is_attributed':'count'})

In [ ]:
mean = mean.merge(count, how='left',left_index=True, right_index=True)
print(mean)

In [16]:
mean.query('count_x > 30').sort_values(by='mean', ascending=False)[0:10]

mean  count_x  count_y
ip     app device os                            
110300 45  1      47  0.500000       32       32
                  43  0.500000       32       32
306868 45  1      40  0.258065       31       31
44744  29  1      19  0.152174       46       46
105560 29  1      19  0.146341       41       41
99856  19  0      0   0.138889       36       36
52094  19  0      38  0.129032       31       31
112302 29  1      19  0.128205       39       39
123994 29  1      19  0.125000       48       48
100275 19  0      21  0.125000       40       40

In [ ]:
joint_col = None
D = 2 ** 26
feature_name_added = 'nc'
training = data
group_by_cols = ['ip','app','device','os']

for col in group_by_cols:
    if joint_col is None:
        joint_col = training[col].astype(str)
    else:
        joint_col = joint_col + "_" + training[col].astype(str)

training['category'] = joint_col.apply(hash) % D
del joint_col

In [19]:
click_buffer = np.full(D, 3000000000, dtype=np.uint32)
training['epochtime'] = training['click_time'].astype(np.int64) // 10 ** 9
next_clicks = []
for category, time in zip(reversed(training['category'].values),
                          reversed(training['epochtime'].values)):
    next_clicks.append(click_buffer[category] - time)
    click_buffer[category] = time
del (click_buffer)
training[feature_name_added] = list(reversed(next_clicks))

In [ ]:
#print(training[['ip','app','device','os','nc']])

a = mean.query('count_x > 30').sort_values(by='mean', ascending=False)[0:10].reset_index()

In [28]:
print('done a')
b = a.merge(training[['ip','app','device','os','nc','click_time', 'is_attributed']], 
            on=['ip','app','device','os'], how='left')

done a


In [29]:
print(b.to_string())

         ip  app  device  os      mean  count_x  count_y          nc          click_time  is_attributed
0    110300   45       1  47  0.500000       32       32        2520 2017-11-09 05:03:48              0
1    110300   45       1  47  0.500000       32       32          69 2017-11-09 05:45:48              0
2    110300   45       1  47  0.500000       32       32         672 2017-11-09 05:46:57              1
3    110300   45       1  47  0.500000       32       32         107 2017-11-09 05:58:09              1
4    110300   45       1  47  0.500000       32       32        1125 2017-11-09 05:59:56              1
5    110300   45       1  47  0.500000       32       32        2485 2017-11-09 06:18:41              1
6    110300   45       1  47  0.500000       32       32        2051 2017-11-09 07:00:06              0
7    110300   45       1  47  0.500000       32       32         678 2017-11-09 07:34:17              1
8    110300   45       1  47  0.500000       32       32        

In [ ]:
a1 = mean.query('mean == 0').sort_values(by='count_x', ascending=False)[0:10].reset_index()
print('done a')
b1 = a1.merge(training[['ip','app','device','os','nc','click_time','is_attributed']], on=['ip','app','device','os'], how='left')

In [33]:
print(b1[8000:10000].to_string())

        ip  app  device  os  mean  count_x  count_y   nc          click_time  is_attributed
8000  5314    3       1  13   0.0     6550     6550    1 2017-11-09 05:29:43              0
8001  5314    3       1  13   0.0     6550     6550    1 2017-11-09 05:29:44              0
8002  5314    3       1  13   0.0     6550     6550    4 2017-11-09 05:29:45              0
8003  5314    3       1  13   0.0     6550     6550    3 2017-11-09 05:29:49              0
8004  5314    3       1  13   0.0     6550     6550    0 2017-11-09 05:29:52              0
8005  5314    3       1  13   0.0     6550     6550    4 2017-11-09 05:29:52              0
8006  5314    3       1  13   0.0     6550     6550    1 2017-11-09 05:29:56              0
8007  5314    3       1  13   0.0     6550     6550    4 2017-11-09 05:29:57              0
8008  5314    3       1  13   0.0     6550     6550    3 2017-11-09 05:30:01              0
8009  5314    3       1  13   0.0     6550     6550    5 2017-11-09 05:30:04    

In [42]:
print(len(training.query('nc > 1489000000')))
print(len(training))
print(len(mean.query('count_x>30')))
print(len(mean.query('count_x==1')))

7684359
37170059
148694
3792049


In [40]:
a2 = mean.query('count_x > 30').sort_values(by='mean', ascending=True)[70000:70010].reset_index()
print('done a')
b2 = a2.merge(training[['ip','app','device','os','nc','click_time','is_attributed']], on=['ip','app','device','os'], how='left')

done a


In [41]:
print(b2[0:1000].to_string())

         ip  app  device   os  mean  count_x  count_y          nc          click_time  is_attributed
0    114276   46       0   24   0.0       31       31       33108 2017-11-09 04:01:05              0
1    114276   46       0   24   0.0       31       31           6 2017-11-09 13:12:53              0
2    114276   46       0   24   0.0       31       31           5 2017-11-09 13:12:59              0
3    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:04              0
4    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:09              0
5    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:14              0
6    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:19              0
7    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:24              0
8    114276   46       0   24   0.0       31       31           5 2017-11-09 13:13:29      

In [18]:
#print(data[1000000:1000003])
data['id']=data.index
data1 = data.set_index('click_time').loc['2017-11-08 04:00:00':'2017-11-08 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

nan
nan
0


82259195
118735619
36476425


In [32]:
data1 = data.set_index('click_time').loc['2017-11-09 04:00:00':'2017-11-09 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

144708152
181878211
37170060


In [22]:
# calculate combined cvr to test hist fts generation

#print('app device cvr:', data[['app','device', 'is_attributed']].groupby(['app','device']).mean().to_string())

os_count = data[['os', 'is_attributed']].groupby(['os']).count()

os_non_attr_count = (1 - data[['os', 'is_attributed']].groupby(['os']).max()) * os_count

print('app device non attri count:', os_non_attr_count.to_string())

#print('app device  attri count:', data[['app','device', 'is_attributed']].groupby(['app','device']).sum().to_string())

#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

app device non attri count:      is_attributed
os                
0                0
1                0
2                0
3                0
4                0
5                0
6                0
7                0
8                0
9                0
10               0
11               0
12               0
13               0
14               0
15               0
16               0
17               0
18               0
19               0
20               0
21               0
22               0
23               0
24               0
25               0
26               0
27               0
28               0
29               0
30               0
31               0
32               0
34               0
35               0
36               0
37               0
38               0
39           94800
40               0
41               0
42               0
43               0
44               0
45               0
46               0
47               0
48               0
49               0
50 

In [24]:

data= data.merge(os_non_attr_count.reset_index(), on=['os'], how='left')


print(data)

               ip  app  device  os  channel          click_time  \
0           83230    3       1  13      379 2017-11-06 14:32:21   
1           17357    3       1  19      379 2017-11-06 14:33:34   
2           35810    3       1  13      379 2017-11-06 14:34:12   
3           45745   14       1  13      478 2017-11-06 14:34:52   
4          161007    3       1  13      379 2017-11-06 14:35:08   
5           18787    3       1  16      379 2017-11-06 14:36:26   
6          103022    3       1  23      379 2017-11-06 14:37:44   
7          114221    3       1  19      379 2017-11-06 14:37:59   
8          165970    3       1  13      379 2017-11-06 14:38:10   
9           74544   64       1  22      459 2017-11-06 14:38:23   
10         172522    3       1  25      379 2017-11-06 14:38:27   
11         105861    3       1  13      379 2017-11-06 14:38:51   
12         210962    3       1  19      379 2017-11-06 14:39:29   
13         124979    3       1  18      379 2017-11-06 14:40:1

In [29]:
xxx = pd.DataFrame(data['is_attributed_y'], dtype='float64')
xxx.to_csv('validation.csv.bz2', index=False, compression='bz2')

In [30]:

log_group = 100000
def rate_calculation(x):
    """Calculate the attributed rate. Scale by confidence"""
    rate = x.sum() / float(x.count())
    conf = np.min([1, np.log(x.count()) / log_group])
    return rate * conf

yyy = data[['app','device', 'is_attributed_x']].groupby(['app','device'])[['is_attributed_x']].apply(rate_calculation).reset_index()
data= data.merge(yyy, on=['app','device'], how='left')

xxx = pd.DataFrame(data['is_attributed_x_y'])
xxx.to_csv('validation1.csv.bz2', index=False, compression='bz2')

In [ ]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()


data.drop('click_time', inplace=True, axis=1)

In [25]:
data0['hour'] = data0["click_time"].dt.hour.astype('uint8')
data0['day'] = data0["click_time"].dt.day.astype('uint8')

data0.drop('click_time', inplace=True, axis=1)

In [15]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')



added hour and day


In [26]:
#print(data.describe())

data0 = data0.query('hour in [{}]'.format(','.join(map(lambda x: str(x), set(data['hour'].unique())))))
print(data0['hour'].unique())

hour_set = set(data['hour'].unique())
hour_set0 = set(data0['hour'].unique())


[14 15  4  5  6  9 10 11 13]


In [27]:
osset = set(data['os'].unique())
osset0 = set(data0['os'].unique())

print(len(osset & osset0))
print(len(osset0))

315
573


In [28]:
chset = set(data['app'].unique())
chset0 = set(data0['app'].unique())

print(len(chset & chset0))
print(len(chset0))

384
610
